In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
print('done')

done


In [2]:
df1= pd.read_csv('/content/fifa.csv') # /content/fifa.csv
#df2= pd.read_csv('/content/fifa_+variable_information.csv')
print(df1.shape)
#print(df2.shape)

FileNotFoundError: [Errno 2] File /content/fifa.csv does not exist: '/content/fifa.csv'

## 1

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
# 2
df_n = pd.concat((df1.iloc[:,[0,1,2,4,6,7,8,10,11,12,13,14,15,16,18,22,25,26]], df1.iloc[:, 26:60]), axis=1)
df_n.shape

In [ ]:
# 3 - Values
import re
df_n['Value'] = df_n['Value'].str.replace('€', '')

In [ ]:
def clrm(n):
  if 'M' in n and '.' in n:
    return float(''.join(re.findall('\d+', n)))*100000
  elif 'M' in n and '.' not in n:
    return float(''.join(re.findall('\d+', n)))*1000000
  elif 'K' in n and '.' in n:
    return float(''.join(re.findall('\d+', n)))*100
  elif 'K' in n and '.' not in n:
    return float(''.join(re.findall('\d+', n)))*1000

In [ ]:

df_n['Value']= df_n['Value'].apply(lambda x: clrm(x) if not isinstance(x, float) else x)

In [ ]:
# Wage
df_n['Wage'] = df_n['Wage'].str.replace('€', '')
df_n['Wage'] = df_n['Wage'].apply(lambda x: clrm(x) if not isinstance(x, float) else x)

In [ ]:
# We now know that we have date in this format as well 
print(max(df1['Joined'], key= lambda x: len(x) if not isinstance(x, float) else 0))

# lets see the head
df1['Joined'].head()

In [ ]:
df1['Joined'] = df1['Joined'].apply(lambda x: int(x.split(' ')[-1]) if not isinstance(x, float) else x)

In [ ]:
# Sanity check with nan
df1['Joined'].isna().sum()

In [ ]:
# Contract valid until 

df_n['Contract Valid Until'].head()

In [ ]:
df_n['Contract Valid Until'] = pd.to_datetime(df1['Contract Valid Until'], errors='coerce')

In [ ]:
# Height
df1['Height'].head()

In [ ]:
df1['Height'].unique()

In [ ]:
df_n['Height']= df1['Height'].apply(lambda x: float('.'.join(x.split("'"))) if not isinstance(x, float) else x)

In [ ]:
# Weight

df1['Weight'].head()

In [ ]:
df_n['Weight'] = df1['Weight'].apply(lambda x: float(x[:-3]) if not isinstance(x, float) else x)

In [ ]:
# Release clause

df_n['Release Clause'].head()

In [ ]:
df_n['Release Clause']= df_n['Release Clause'].apply(lambda x: clrm(x) if not isinstance(x, float) else x)

In [ ]:
df_n['Release Clause'].isna().sum()

In [ ]:
# mean imputations

for i in df_n.select_dtypes(include='number'):
  df_n[i] = df_n[i].fillna(df_n[i].mean())

In [ ]:
# overall ratings

sns.distplot(df1['Overall'])

In [ ]:
df1['Overall'].skew()

In [ ]:
#1
sns.boxplot(df1['Overall'])

In [ ]:
# right skewed distribution
# most players are in 70-80 ratings
# few in 90-95 ratings

In [ ]:
r= ['Overall', 'Value', 'Wage', 'International Reputation', 'Height', 'Weight', 'Release Clause']

In [ ]:
#2
sns.pairplot(df_n[r])

In [ ]:
#3

av= df_n.loc[(df_n['Contract Valid Until'] == '2020-01-01T00:00:00.000000000')].sort_values(by='Overall', ascending=False)[:20]

In [ ]:
av.head()

In [ ]:
#3 a
av['Wage'].mean()

In [ ]:
av['Age'].mean()

In [ ]:
# C
sns.regplot(x='Overall', y='Value', data= df_n)

In [ ]:
# Yes there is a strong correlation between value and overall rating

In [ ]:
#4

top_5 = {}
for i in df_n['Position'].unique():
  try:
    top_5[i]=list(df_n.sort_values(by=['Overall','Position'], ascending=[False, True]).groupby(df_n['Position']).get_group(i)[:5]['Name'].values)
  except KeyError:
    pass

In [ ]:
len(top_5.keys())

In [ ]:
len(df_n['Position'].unique())

In [ ]:
# We have one less value bcoz of one NaN

In [ ]:
df_top_5 = pd.DataFrame(top_5)
df_top_5

In [ ]:
#4a

df_top_5

In [ ]:
from collections import Counter
dl= []
for i in top_5.values():
  for j in i:
    if not isinstance(j, float):
      dl.append(j)

In [ ]:
Counter(dl).most_common()[:2]

In [ ]:
# There are no players being repeated. 

In [ ]:
# b

sal= {}
for i in df_top_5.columns:
  h= []
  for j in df_top_5[i]:
    try:
      h.append(*df_n.loc[df_n['Name'] == j]['Wage'].values)
    except TypeError:
      pass
  sal[i]= sum(h)

In [ ]:
# these are the average wage one can expect to pay position wise top 5 players.

sal

In [ ]:
# sanity check
len(sal.keys())

In [ ]:
# Players recommended to the Brussels United FC

for i in df_top_5.columns:
  print(df_top_5[i][0])

In [ ]:
df_n.shape